# About dataset

#### This dataset consists of reviews of fine foods from amazon. <br>The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. <br> It also includes reviews from all other Amazon categories. <br><br>Data set: https://www.kaggle.com/snap/amazon-fine-food-reviews

#### Dataset info:

Number of reviews: 568,454 <br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10<br><br>

<b>Attribute Information:</b><br>

Id - unique identifier for the review<br>
ProductId - unique identifier for the product<br>
UserId - unqiue identifier for the user<br>
ProfileName<br>
HelpfulnessNumerator - number of users who found the review helpful<br>
HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not<br>
Score - rating between 1 and 5<br>
Time - timestamp for the review<br>
Summary - brief summary of the review<br>
Text - text of the review<br>

<br>
The dataset is .sqlite file

In [36]:
# Imports

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 
import re

In [4]:
# Load the dataset using sqlite

con = sqlite3.connect('database.sqlite')

original_data = pd.read_sql_query(""" SELECT * FROM Reviews """, con)

original_data.shape

(568454, 10)

# Objective

#### Our objective is to determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2) given a review

Since we want to predict the reviews which are either positive or negative, we can drop the data where 'Score' is 3. <br>
We will also change 'Score' column logic to indicate either positive or negative and predict this ourselves based on sentiment in 'Text' column.

In [19]:
filtered_data = original_data[original_data['Score']!=3]
filtered_data.drop('Score', axis=1)


# Lets replace 'Score' as follows 
def partition(x):
    if x > 3:
        return 1
    return 0

actualScores = filtered_data['Score']
positiveNegative = actualScores.map(partition)
filtered_data['Score'] = positiveNegative
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Data Cleaning

In [20]:
cleaning= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
cleaning.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


<b>Observation 1:</b> We observed that when a product has variations, amazon combines the reviews for all the variations. Since it make sense to drop the variation product reviews and keep distinct ones. The reason being, it will be very easy for our ML model to predict the review of product variations based on original product review. <b> Lets remove the duplicates </b>

In [21]:
# Lets sort data based on ProductId in ascending order
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [23]:
# Lets do deduplciation
final = sorted_data.drop_duplicates(subset={"UserId", "ProfileName","Time", "Text"}, keep="first", inplace=False)

final.shape

(364173, 10)

In [24]:
# Check how much data is preserved after deduplication
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

<b>Observation 2: </b>We know that 'HelpfulnessNumerator' should be less than 'HelpfulnessDenominator'.<br>Few rows violet this. <b>Lets drop 'em.</b>

In [29]:
final[final['HelpfulnessNumerator'] > final['HelpfulnessDenominator']]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,1,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,1,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [30]:
final = final[final['HelpfulnessNumerator'] <= final['HelpfulnessDenominator']]

In [32]:
final.shape

(364171, 10)

In [33]:
final['Score'].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

 <b>Observation 3: </b>We observed that the food reviews data has some reviews of books. We can use 'Text' & 'Summary' columns to identify whether they are book reviews if words like 'book', 'read', 'reading' are there.<br>
 There is also a possibility that some food reviews contain words like these. It's upto us whether to delete this data or not. Since we got this data from data team, we will consider that these reviews are for food & not for book.<br>
 <b>If there will be need to delete this data, just make below cell as 'Code' and run.</b>

def apply_mask_summary(data,regex_string):
    mask = data.Summary.str.lower().str.contains(regex_string)
    data.drop(data[mask].index, inplace=True)

def apply_mask_text(data,regex_string):
    mask = data.Text.str.lower().str.contains(regex_string)
    data.drop(data[mask].index, inplace=True)


apply_mask_summary(final,re.compile(r"\bbook\b"))
apply_mask_summary(final,re.compile(r"\bread\b"))

apply_mask_text(final,re.compile(r"\bbook\b"))
apply_mask_text(final,re.compile(r"\bread\b"))

apply_mask_summary(final,re.compile(r"\bbooks\b"))
apply_mask_summary(final,re.compile(r"\breads\b"))

apply_mask_text(final,re.compile(r"\bbooks\b"))
apply_mask_text(final,re.compile(r"\breads\b"))

apply_mask_summary(final,re.compile(r"\breading\b"))
apply_mask_text(final,re.compile(r"\breading\b"))